In [1]:
%%capture
!pip install python-docx
!pip install PyPDF2
!pip install aspose-words

In [2]:
from __future__ import annotations
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import docx
import pandas as pd
import numpy as np
import json
import aspose.words as aw
import traceback
import re


print('docx version:', docx.__version__)
print('PyPDF2 version:',PyPDF2.__version__)

docx version: 0.8.11
PyPDF2 version: 2.12.1


# Load data

In [3]:
import gdown

url = 'https://drive.google.com/file/d/1_E3GLMkbboLZcUPVfQHw99RWU11znbAq/view?usp=share_link'
file_name = 'data'
gdown.download(url, file_name, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1_E3GLMkbboLZcUPVfQHw99RWU11znbAq
To: /content/data
100%|██████████| 9.68M/9.68M [00:00<00:00, 43.9MB/s]


'data'

In [4]:
%%capture
!unzip '/content/data' -d '/content/data_dir'

In [5]:
data_dir = '/content/data_dir/hacka-aka-embedika-2/docs'
json_path = '/content/data_dir/hacka-aka-embedika-2/classes.json'

converted_docx_dir = 'converted_docx'
filenames = os.listdir(data_dir)

In [6]:
with open(json_path) as json_file:
    label_dict = json.load(json_file)

In [7]:
def init_file_paths(data_dir_path: str) -> tuple[list[tuple[str,str]],
                                                 list[tuple[str,str]],
                                                 list[tuple[str,str]],
                                                 list[tuple[str,str]]]:
    '''
        Создаем списоки кортежей (полный путь, имя файла) для каждого расширения файла
    '''
    doc_file_paths = []
    docx_file_paths = []
    pdf_file_paths = []
    rtf_file_paths = []
    for filename in os.listdir(data_dir_path):
        if filename.endswith('.docx', -5):
            docx_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.pdf', -5):
            pdf_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.rtf', -5):
            rtf_file_paths.append((os.path.join(data_dir_path,filename), filename))
        elif filename.endswith('.doc', -5):
            doc_file_paths.append((os.path.join(data_dir_path,filename), filename))
    
    print(f'Количество pdf-файлов: {len(pdf_file_paths)}')
    print(f'Количество doc-файлов: {len(doc_file_paths)}')
    print(f'Количество docx-файлов: {len(docx_file_paths)}')
    print(f'Количество rtf-файлов: {len(rtf_file_paths)}')
    
    return doc_file_paths, docx_file_paths, pdf_file_paths, rtf_file_paths

doc_file_paths, docx_file_paths, pdf_file_paths, rtf_file_paths = init_file_paths(data_dir)
files_paths_to_docx = []
files_paths_to_docx += doc_file_paths
files_paths_to_docx += rtf_file_paths


Количество pdf-файлов: 13
Количество doc-файлов: 81
Количество docx-файлов: 22
Количество rtf-файлов: 4


In [8]:
def convert_doc_docx(doc_file_paths: str, converted_files_dir_path: str)->list[tuple[str,str]]:
    '''
        Конвертируем .doc файлы в .docx, так как python не открывает файлы .doc
        Возвращаемые кортежи содержат правилльный полный путь к файлу, но старое расширение в имени файла
        Новые файлы содержат в начале Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.
    '''
    new_docx_paths = []
    file_extension = re.compile(r'\..*')
    for doc_file_path, doc_file_name in doc_file_paths:
        doc = aw.Document(doc_file_path)
        new_docx_path = os.path.join(converted_files_dir_path, file_extension.sub("", doc_file_name) + '.docx')
        doc.save(new_docx_path)
        new_docx_paths.append((new_docx_path, doc_file_name))
    return new_docx_paths
docx_file_paths += convert_doc_docx(files_paths_to_docx, converted_docx_dir)
print(f'Количество pdf-файлов: {len(pdf_file_paths)}')
print(f'Количество docx-файлов: {len(docx_file_paths)}')

Количество pdf-файлов: 13
Количество docx-файлов: 107


In [9]:
def read_docx_file(docx_file_path: str) -> str:
    '''
        Открываем .docx файл и, соединяя все абзацы,
        возращем полный текст документа
        или None если не удалось открыть файл
    '''
    try:
        doc = docx.Document(docx_file_path)
        all_paras = doc.paragraphs
        full_doc_text = ''
        for para in all_paras:
            full_doc_text = full_doc_text + '\n' + para.text
        return full_doc_text.replace('Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.', '')
    except:
        print(f'Ошибка: Не удалось открыть файл: {docx_file_path}\n {traceback.format_exc()}')
        return None

def read_pdf_file(pdf_file_path: str) -> str:
    '''
        Открываем .pdf файл и, соединяя все страницы,
        возращем полный текст документа
        или None если не удалось открыть файл
    '''
    try:
        full_pdf_text = ''
        with open(pdf_file_path, "rb") as filehandle:
            pdf = PdfFileReader(filehandle)
            num_pages = pdf.getNumPages()
            for num in range(num_pages):
                page = pdf.getPage(num)
                full_pdf_text = full_pdf_text + '\n' + page.extractText()
        return full_pdf_text
    except:
        print(f'Ошибка: Не удалось открыть файл: {pdf_file_path}\n {traceback.format_exc()}')
        return None

In [10]:
def get_document_text(document_file_path: str) -> str:
    re_file_extension = re.compile(r'\.\w+')

    file_extension = re_file_extension.findall(document_file_path)
    if file_extension[0] == '.docx':
        return read_docx_file(document_file_path)
    
    elif file_extension[0] == '.doc' or file_extension[0] == '.rtf':
        doc = aw.Document(document_file_path)
        new_docx_path = re_file_extension.sub('',docx_file_paths[0][0]) + '.docx'
        doc.save(new_docx_path)
        return read_docx_file(new_docx_path)

    elif file_extension[0] == '.pdf':
        return read_pdf_file(document_file_path)

In [11]:
def create_df(pdf_file_paths: list[str], docx_file_paths: list[tuple[str, str]], label_dict: dict[str: str]):
    text_label_filename = []
    for docx_file_path, docx_file_name in docx_file_paths:
        text_label_filename.append([read_docx_file(docx_file_path),
                                    label_dict.get(docx_file_name, None), docx_file_name])
    for pdf_file_path, pdf_file_name in pdf_file_paths:
        text_label_filename.append([read_pdf_file(pdf_file_path),
                                    label_dict.get(pdf_file_name, None), pdf_file_name])
    
    df = pd.DataFrame(data=text_label_filename, columns=['text', 'label', 'doc_file_name'])
    return df
df = create_df(pdf_file_paths, docx_file_paths, label_dict)
df.dropna(inplace=True)
df = df.reset_index(drop=True)
damaged_files_names = ['a2f8675b6d37b6af819ca3fdbb0e3432.pdf', '2054a58154349bc54b691666fe3f89b1.pdf']
ind_of_damadged_data = df[df['doc_file_name'].apply(lambda x: x in damaged_files_names)].index
df = df.drop(index = ind_of_damadged_data).reset_index(drop=True)
df = df[df['text'].apply(lambda x: len(x.split()) > 10 )]
df.to_csv('text_data.csv', index=False)

Ошибка: Не удалось открыть файл: /content/data_dir/hacka-aka-embedika-2/docs/18d573815c15b4e798bdfbfb52fb2f43.docx
 Traceback (most recent call last):
  File "<ipython-input-9-b112943da1f6>", line 8, in read_docx_file
    doc = docx.Document(docx_file_path)
  File "/usr/local/lib/python3.8/dist-packages/docx/api.py", line 25, in Document
    document_part = Package.open(docx).main_document_part
  File "/usr/local/lib/python3.8/dist-packages/docx/opc/package.py", line 130, in open
    Unmarshaller.unmarshal(pkg_reader, package, PartFactory)
  File "/usr/local/lib/python3.8/dist-packages/docx/opc/package.py", line 198, in unmarshal
    parts = Unmarshaller._unmarshal_parts(
  File "/usr/local/lib/python3.8/dist-packages/docx/opc/package.py", line 215, in _unmarshal_parts
    parts[partname] = part_factory(
  File "/usr/local/lib/python3.8/dist-packages/docx/opc/part.py", line 191, in __new__
    return PartClass.load(partname, content_type, blob, package)
  File "/usr/local/lib/python3.8

In [13]:
loaded_df = pd.read_csv('/content/text_data.csv')
loaded_df

,text,label,doc_file_name
0,\nДоговор возмездного оказания услуг № {НомерД...,Договоры для акселератора/Договоры оказания услуг,317e4552102e82bd9e17ba1205cd4367.docx
1,\nДОГОВОР \n\n№ ______ от «___» ___________ __...,Договоры для акселератора/Договоры подряда,03a70fe60be9ecc2a63798a361fc9689.docx
2,\nДоговор \n \nг. Новосибирск ...,Договоры для акселератора/Договоры купли-продажи,8c5ce9a193c788cadf94ddec5607f3b4.docx
3,\nДоговор № {НомерДокумента}\n{НазваниеКонтр}...,Договоры для акселератора/Договоры поставки,84fec112d02288861e7af59f468131fb.docx
4,\nУтверждена\nприказом Министерства образовани...,Договоры для акселератора/Договоры оказания услуг,0b4be82b86eff410d69d1d6b5553d220.docx
...,...,...,...
106,\nДОГОВОР ПОСТАВКИ № _________________________...,Договоры для акселератора/Договоры поставки,214d620d9c54bc83111277dd872d3cb2.pdf
107,\n(Типовая форма) \nДОГОВОР КУПЛИ -ПРОДАЖИ №_...,Договоры для акселератора/Договоры купли-продажи,086ad48895d2a73854b6151decc28800.pdf
108,\n1 \n \n ДОГОВОР ОКАЗАНИЯ УСЛУГ № __/________...,Договоры для акселератора/Договоры оказания услуг,736cc87395089b9611b0db70d2999d75.pdf
109,\nДОГОВОР № \nкупли-продажи\nг. Москва ...,Договоры для акселератора/Договоры купли-продажи,4a5707e447271a188a1211606b158a94.pdf
